In [1]:
'''
Conditions on the clusters (Y)_s: 

1, Every v has its r-nbhd completely contained in EXACTLY one cluster. 
2. Overlap #define as max no' of clusters that contain a single v, less than beta times the beta'th root of n 
3. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  
4. All the kers are disjoint and yet cover the entire set.

Invariants through _a single phase_

1. All Ys in a phase disjoint. 
2. All kerZs in a phase disjoint.
3. Z can overlap. 
4. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  
5. The union of all Ys is a subset of the union of all kerZs . 
6. The ker(Z), Z relationship and the ker(Y), Y relationship are maintained -- as well as the Y_s, kerZ_s relationship mentioned above.

'''

"\nConditions on the clusters (Y)_s: \n\n1, Every v has its r-nbhd completely contained in EXACTLY one cluster. \n2. Overlap #define as max no' of clusters that contain a single v, less than beta times the beta'th root of n \n3. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  \n4. All the kers are disjoint and yet cover the entire set.\n\nInvariants through _a single phase_\n\n1. All Ys in a phase disjoint. \n2. All kerZs in a phase disjoint.\n3. Z can overlap. \n4. For every element in some ker(~) the r-nbhd of that element is contained in ~ .  \n5. The union of all Ys is a subset of the union of all kerZs . \n6. The ker(Z), Z relationship and the ker(Y), Y relationship are maintained -- as well as the Y_s, kerZ_s relationship mentioned above.\n\n"

In [2]:
import sys 
sys.path.append('./lib') 

In [3]:
import networkx as nx 
import plotly.graph_objects as go 
from lib.graphfig import * 
from lib.bfs import * 
from lib.even_shiloach import *

In [ ]:
def functional_cluster_gen(R:set[int],U:set[int],v:int,Y_s:set[set[int]],G:nx.Graph,r:int,beta:int|float):
    print("0") 
    n = len(G.nodes)
    print("1")
    if len(Y_s) == 0:
        G_notin_Y = G.copy()
    else:
        G_notin_Y = nx.induced_subgraph(G,set(G.nodes)-set.union(*Y_s))
    print("2")
    ker_z = set([v,])
    T_z = BFSAlgo(G,ker_z,r)
    print("3")
    z = set(T_z.bfs_graph.nodes)
    # try:
    for v_z in z:  
        assert T_z.bfs_graph.nodes[v_z]['level'] <= r 
    print("4")
    # except AssertionError: 
    #     print("Assertion Error")
    #     print(T_z.bfs_graph.nodes[v_z])
    #     print(r)
    #     print(z)
    #     print(T_z.bfs_graph.nodes)
    #     print(T_z.bfs_graph.edges)
    #     print(T_z.bfs_graph.nodes[v_z])
    #     print(T_z.bfs_graph.nodes[v_z]['level'])
    #     print(T_z.bfs_graph.nodes[v_z]['level'] <= r)
        # yield T_z

    cluster_iters = 0
    while True: 
        print("L1")
        ker_y, y = ker_z.copy(), z.copy()
        T_z = BFSAlgo(G_notin_Y,y,2*r)
        print("L2")
        z = set(T_z.bfs_graph.nodes)
        ker_z = set([v for v in U.intersection(z) if  T_z.bfs_graph.nodes[v]['level']  <= r ]) 
        print("L3")
        
        n_root_beta = n**(1/beta) 
        lenR_root_beta = len(R)**(1/beta)

        A1 = len(z)
        A2 = n_root_beta*len(y) 
        B1 = len(ker_z) 
        B2 = len(ker_y)*lenR_root_beta
        C1 = sum(dict(G.degree(z)).values()) 
        C2 = sum(dict(G.degree(y)).values())*n_root_beta 

        print("L3")
        
        print(f"Cluster Iteration {cluster_iters}:")
        print(f"len(z) = {A1}")
        print(f"len(y) = {len(y)}") 
        print(f"A1 = {round(A1,2)}, A2 = {round(A2,2)}, B1 = {round(B1,2)}, B2 = {round(B2,2)}, C1 = {round(C1,2)}, C2 = {round(C2,2)}\n\n")
        # yield None 
        if(A1 <= A2 and B1 <= B2 and C1 <= C2):
            # yield True 
            break 
        cluster_iters += 1
    return (ker_y, y, ker_z, z)






In [184]:
for p_search in reversed(range(1,31)):
    
    base_graph =  nx.fast_gnp_random_graph(10000,p_search*1e-4)
    try:
        assert nx.is_connected(base_graph)
    except AssertionError:
        print(p_search)

7
6
5
4
3
2
1


In [72]:
def get_connected_gnp_graph(n, lower_bound_n, p): 
    pre_base_graph =  nx.fast_gnp_random_graph(n,p)
    cc_nodes_lst = list(nx.connected_components(pre_base_graph))
    cc_lens_lst = [len(i) for i in cc_nodes_lst]
    idx = cc_lens_lst.index(max(cc_lens_lst)) 
    base_graph = nx.induced_subgraph(pre_base_graph,cc_nodes_lst[idx]).copy()
    try:
        assert len(base_graph.nodes) > lower_bound_n
    except AssertionError:
        print(len(base_graph.nodes)) 
        print([len(i) for i in nx.connected_components(pre_base_graph)])
        raise AssertionError

    return base_graph

In [73]:
[len(i) for i in list(nx.connected_components(base_graph))]

[3814]

In [88]:
from tqdm import tqdm

In [91]:
for _ in tqdm(range(100)):
    base_graph = get_connected_gnp_graph(20000,5000,0.00006)

100%|██████████| 100/100 [00:11<00:00,  8.79it/s]


In [96]:
R = set(base_graph.nodes)
U = R.copy()
v = random.choice(list(base_graph.nodes))
Y_s = []
G = base_graph
r = 4
beta = 5
print(len(base_graph.nodes))
out = functional_cluster_gen(R,U,v,Y_s,G,r,beta)

6134
0
1
2
3
4
L1
L2
L3
L3
Cluster Iteration 0:
len(z) = 155
len(y) = 17
A1 = 155, A2 = 97.27, B1 = 59, B2 = 5.72, C1 = 357, C2 = 223.16


L1
L2
L3
L3
Cluster Iteration 1:
len(z) = 847
len(y) = 155
A1 = 847, A2 = 886.91, B1 = 393, B2 = 337.6, C1 = 1866, C2 = 2042.76


L1
L2
L3
L3
Cluster Iteration 2:
len(z) = 2864
len(y) = 847
A1 = 2864, A2 = 4846.54, B1 = 1668, B2 = 2248.75, C1 = 6152, C2 = 10677.27




0
1
2
3
4
L1
L2
L3
L3
Cluster Iteration 0:
len(z) = 347
len(y) = 13
A1 = 347, A2 = 136617.0, B1 = 79, B2 = 10509.0, C1 = 1183, C2 = 472905.0




In [158]:
ker_y, y, ker_z, z = out